In [42]:
import pandas as pd
import os
dateFile_p  = 'E:/sp2023stock/TopicModeling/data/ModelingDataset/Test/V.2/dateonly.csv'
TopdocFile_p =  'E:/sp2023stock/TopicModeling/result/modelOutput/document_topics.csv'
labeled_topics_path = 'E:/sp2023stock/TopicModeling/result/modelOutput/labeled_topics.csv'

#### News LDA (KIM)

In [ ]:
df1 = pd.read_csv(dateFile_p)
df2 = pd.read_csv(TopdocFile_p)

print(f"Date File = {len(df1)}, Data File = {len(df2)}")

df_merged = pd.concat([df1, df2], axis=1)
df_merged.drop(['ID', 'Agency'], axis=1, inplace=True)

print(f" Merged data = {len(df_merged)}")

In [ ]:
# Assuming 'result' is your DataFrame and you've already executed:
result = df_merged.groupby('Date').sum().reset_index()
result['Date'] = pd.to_datetime(result['Date']).dt.date

# Define the date range scope
start_date = pd.to_datetime('2018-01-01').date()  # Use .date() to match the format in 'Date' column
end_date = pd.to_datetime('2023-12-31').date()

# Generating a complete list of dates within the defined range
all_dates = pd.date_range(start_date, end_date).date  # Convert to datetime.date for consistency

# Convert 'Date' from datetime.date objects to Timestamps for reindexing
result['Date'] = pd.to_datetime(result['Date'])

# Set 'Date' as index
result.set_index('Date', inplace=True)

# Reindex with the complete date range (convert all_dates to Timestamps to match index type)
result = result.reindex(pd.to_datetime(all_dates), fill_value=0)


result.reset_index(inplace=True)
result.rename(columns={'index': 'Date'}, inplace=True)
result['Date'] = result['Date'].dt.date  # Convert Timestamps back to dates
result=result.round(8)
result.to_csv('E:/sp2023stock/TopicModeling/result/integrationOutput/Kim/News.csv', index=False)


##### Select top 10 value

In [18]:

df = pd.read_csv('E:/sp2023stock/TopicModeling/result/integrationOutput/Kim/News.csv', nrows=5)
topic_columns = [col for col in df.columns if col.startswith('Topic_')]

for col in topic_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Iterate over the DataFrame rows
for index, row in df.iterrows():
    # Get the topic columns for the current row, drop missing values, and find the indices of the top 3 values
    top_values = row[topic_columns].dropna().nlargest(10)
    top_indices = top_values.index
    # Set all other values to 0
    for col in topic_columns:
        if col not in top_indices:
            df.at[index, col] = 0


TypeError: Cannot use method 'nlargest' with dtype object

##### LDA-400 Econ (Filtered)
เลือกข่าวเฉพาะที่มี labeled topic มาใช้
<br>
โดยเลือกจาก topic ที่อยู่ใน top 10 ของข่าวนั้นๆ
ตัวอย่าง -> 41245,Topic_1,Topic_91,Topic_142

In [69]:
import pandas as pd

df = pd.read_csv(TopdocFile_p)
datedf = pd.read_csv(dateFile_p)
labeled_topics_df = pd.read_csv(labeled_topics_path)
labeled_topics = labeled_topics_df ['Topic'].tolist()

def find_top_3_topics(row):
    top_3 = row.nlargest(10)
    return pd.Series([top_3.index.tolist(), top_3.values.tolist()], index=['Top_3_Topics', 'Top_3_Values'])

top_3_df = df.apply(find_top_3_topics, axis=1)
max_top_topics = top_3_df['Top_3_Topics'].apply(len).max()
dynamic_columns = [f'Top_{i+1}' for i in range(max_top_topics)]
top_3_topics_df = pd.DataFrame(top_3_df['Top_3_Topics'].tolist(), columns=dynamic_columns)

merged_result = pd.concat([datedf, top_3_topics_df], axis=1)
filtered_topic = merged_result

filtered_topic['ECON'] = filtered_topic.apply(lambda row: any(row.get(f'Top_{i}', None) in labeled_topics for i in range(1, 10)), axis=1)
filtered_topic_result = filtered_topic[['ID', 'Agency', 'Date', 'ECON']]
topic_with_filtered = pd.concat([filtered_topic_result, df], axis=1)

#topic_with_filtered.to_csv('E:/sp2023stock/TopicModeling/result/integrationOutput/Kim/filtered_News.csv', index=False)
topic_columns = [col for col in topic_with_filtered.columns if col.startswith('Topic_')]  # List your topic columns here
topic_with_filtered.loc[topic_with_filtered['ECON'] == False, topic_columns] = 0

topic_with_filtered.drop(['ECON','ID'], axis=1, inplace=True)

####################################################################################
df_merged = topic_with_filtered.copy()
####################################################################################

# Summarize topic distributions by Date

#result = df_merged.groupby('Date').sum().reset_index()
result = df_merged.groupby('Date').sum(numeric_only=True).reset_index()
result['Date'] = pd.to_datetime(result['Date']).dt.date

start_date = pd.to_datetime('2018-01-01').date()  # Use .date() to match the format in 'Date' column
end_date = pd.to_datetime('2023-12-31').date()
all_dates = pd.date_range(start_date, end_date).date  # Convert to datetime.date for consistency


result['Date'] = pd.to_datetime(result['Date'])
result.set_index('Date', inplace=True)
result = result.reindex(pd.to_datetime(all_dates), fill_value=0)


result.reset_index(inplace=True)
result.rename(columns={'index': 'Date'}, inplace=True)
result['Date'] = result['Date'].dt.date  # Convert Timestamps back to dates
result=result.round(8)
result.to_csv('E:/sp2023stock/TopicModeling/result/integrationOutput/Kim/filtered_News1.csv', index=False)



#### Twitter LDA

In [ ]:
import pandas as pd
import os
raw_twitter = 'E:/sp2023stock/TopicModeling/data/twitter/topic_prob.xlsx'
df_raw_twitter = pd.read_excel(raw_twitter)
len(df_raw_twitter)

In [ ]:
info_twitter = 'E:/sp2023stock/TopicModeling/data/twitter/CleanTwitterData.parquet'
df_info_twitter = pd.read_parquet(info_twitter)
df_info_twitter.drop(['author_id',
 'edit_history_tweet_ids',
 'id',
 'text',
 'aspect',
 'clean text',
 'economic_topic',
 'clean_neg',
 'clean_neu',
 'clean_pos',
 'clean_q',
 'clean_preds'], axis=1, inplace=True)
df_info_twitter

In [ ]:
import pandas as pd

# Convert 'created_at' to datetime
df_info_twitter['created_at'] = pd.to_datetime(df_info_twitter['created_at'])
# Option 1: Extract just the date part as a datetime.date object
df_info_twitter['date_only'] = df_info_twitter['created_at'].dt.date
df_info_twitter['date_only']


In [ ]:
twitter_merged_result = pd.concat([df_info_twitter['date_only'],df_raw_twitter], axis=1)
twitter_merged_result.drop('tweet_id', axis=1, inplace=True)
twitter_merged_result.rename(columns={'date_only': 'Date'}, inplace=True)
twitter_merged_result

In [ ]:
# Assuming 'result' is your DataFrame and you've already executed:
result_twitter = twitter_merged_result.groupby('Date').sum().reset_index()
result_twitter['Date'] = pd.to_datetime(result_twitter['Date']).dt.date

# Define the date range scope
start_date = pd.to_datetime('2018-01-01').date()  # Use .date() to match the format in 'Date' column
end_date = pd.to_datetime('2023-12-31').date()

# Generating a complete list of dates within the defined range
all_dates = pd.date_range(start_date, end_date).date  # Convert to datetime.date for consistency

# Convert 'Date' from datetime.date objects to Timestamps for reindexing
result_twitter['Date'] = pd.to_datetime(result_twitter['Date'])

result_twitter.set_index('Date', inplace=True)

result_twitter = result_twitter.reindex(pd.to_datetime(all_dates), fill_value=0)

result_twitter.reset_index(inplace=True)
result_twitter.rename(columns={'index': 'Date'}, inplace=True)
result_twitter['Date'] = result_twitter['Date'].dt.date  # Convert Timestamps back to dates
result_twitter=result_twitter.round(8)
result_twitter.to_csv('E:/sp2023stock/TopicModeling/result/integrationOutput/Kim/Twitter.csv', index=False)

##### GDELT

In [14]:
import pandas as pd

# Path to your CSV file
csv_file_path = 'E:/sp2023stock/TopicModeling/data/GDELT/GDELT2.csv'
gdelt_v1_df = pd.read_csv(csv_file_path)

gdelt_v1_df['Date'] = pd.to_datetime(gdelt_v1_df['Date'], format='%Y%m%d')
# Convert back to a string in the new format
gdelt_v1_df['Date'] = gdelt_v1_df['Date'].dt.strftime('%Y-%m-%d')
gdelt_v1_df.fillna(0, inplace=True)
gdelt_v1_df['Date'] = pd.to_datetime(gdelt_v1_df['Date'])

result_gdelt_v1 = gdelt_v1_df.groupby('Date').sum().reset_index()
result_gdelt_v1['Date'] = pd.to_datetime(result_gdelt_v1['Date'], errors='coerce').dt.date

# Define the date range scope
start_date = pd.to_datetime('2018-01-01').date()  # Use .date() to match the format in 'Date' column
end_date = pd.to_datetime('2023-12-31').date()

# Generating a complete list of dates within the defined range
all_dates = pd.date_range(start_date, end_date).date  # Convert to datetime.date for consistency

# Convert 'Date' from datetime.date objects to Timestamps for reindexing
result_gdelt_v1['Date'] = pd.to_datetime(result_gdelt_v1['Date'])

result_gdelt_v1.set_index('Date', inplace=True)

result_gdelt_v1 = result_gdelt_v1.reindex(pd.to_datetime(all_dates), fill_value=0)

result_gdelt_v1.reset_index(inplace=True)
result_gdelt_v1.rename(columns={'index': 'Date'}, inplace=True)
# result_gdelt_v1['Date'] = result_gdelt_v1['Date'].dt.date  # Convert Timestamps back to dates
result_gdelt_v1 = result_gdelt_v1.round(7)
result_gdelt_v1.to_csv('E:/sp2023stock/TopicModeling/result/integrationOutput/Kim/gdelt_v2.csv', index=False)
result_gdelt_v1


,Date,ECON_CENTRALBANK,ECON_CURRENCY_EXCHANGE_RATE,ECON_CURRENCY_RESERVES,ECON_DEBT,ECON_DIESELPRICE,ECON_ELECTRICALDEMAND,ECON_ELECTRICALGENERATION,ECON_ELECTRICALGRID,ECON_ELECTRICALPRICE,...,WB_1998_WATER_ECONOMICS,WB_2111_COMPETITION_ECONOMICS,WB_427_WATER_ALLOCATION_AND_WATER_ECONOMICS,WB_439_MACROECONOMIC_AND_STRUCTURAL_POLICIES,WB_440_MACROECONOMIC_MONITORING,WB_471_ECONOMIC_GROWTH,WB_625_HEALTH_ECONOMICS_AND_FINANCE,WB_702_ECONOMIC_SHOCKS_AND_VULNERABILITY,WB_836_POLITICAL_ECONOMY_OF_REFORM,WB_862_GROWTH_POLES_AND_ECONOMIC_ZONES
0,2018-01-01,0.0,0.0000,0.0,-1.1773,0.0000,0.0,0.0000,0.0,0.0,...,0.0000,0.0,0.0000,0.0000,0.0,-2.1125,0.0000,0.0,0.0000,0.0000
1,2018-01-02,0.0,-1.0022,0.0,-2.0066,0.0000,0.0,1.4675,0.0,0.0,...,0.0000,0.0,0.0000,0.2485,0.0,2.9138,0.0000,0.0,-2.4254,-6.9257
2,2018-01-03,0.0,1.4469,0.0,-0.9472,2.9851,0.0,0.0000,0.0,0.0,...,3.4247,0.0,2.3642,0.0000,0.0,-0.6669,0.0000,0.0,0.0000,-6.8129
3,2018-01-04,0.0,0.9571,0.0,-0.5903,2.5763,0.0,0.0000,0.0,0.0,...,1.2276,0.0,-0.6581,0.0000,0.0,-0.6927,0.0000,0.0,0.0000,-4.8980
4,2018-01-05,0.0,0.0000,0.0,-1.1658,0.0000,0.0,0.0000,0.0,0.0,...,0.0000,0.0,-0.2566,0.0000,0.0,-0.8980,0.0000,0.0,-1.7150,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2186,2023-12-27,0.0,-1.4070,0.0,-0.4544,0.0000,0.0,0.0000,0.0,0.0,...,0.0000,0.0,0.0000,0.3278,0.0,3.9519,-1.5827,0.0,0.0000,0.0000
2187,2023-12-28,0.0,0.0000,0.0,1.8940,0.0000,0.0,0.0000,0.0,0.0,...,0.0000,0.0,-7.9060,0.6211,0.0,0.4188,1.2136,0.0,0.0000,0.0000
2188,2023-12-29,0.0,-3.7288,0.0,-1.4406,0.0000,0.0,0.0000,0.0,0.0,...,0.0000,0.0,2.0642,-0.4989,0.0,2.4297,0.6009,0.0,0.0000,0.0000
2189,2023-12-30,0.0,-2.4085,0.0,-2.4422,0.0000,0.0,0.0000,0.0,0.0,...,0.0000,0.0,0.0000,2.1185,0.0,1.7563,0.0000,0.0,0.0000,0.0000
